# Transformación de tablas

Llamos a los dataframes para poder utilizarlos

In [156]:
%store -r apple_price
%store -r microsoft_price
%store -r twitter_price
%store -r apple_earnings
%store -r microsoft_earnings
%store -r twitter_earnings

In [98]:
%pip install joblib

     |████████████████████████████████| 306 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [99]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from joblib import Parallel, delayed

In [153]:
from apifunctions import *

## Tablas earnings

A estas tablas queremos quitarles el símbolo del dolar a los valores de EPS y queremos establecer los tipos de datos de las columnas

earnings_list=[apple_earnings,microsoft_earnings,twitter_earnings]
for i in earnings_list:
    quitar_dolar(i)

In [161]:
for i in range(len(apple_earnings["Estimated EPS"])):
    
        apple_earnings["Estimated EPS"][i]=apple_earnings["Estimated EPS"][i].replace("$","") 
        apple_earnings["Actual EPS"][i]=apple_earnings["Actual EPS"][i].replace("$","") 

In [157]:
for i in range(len(microsoft_earnings["Estimated EPS"])):
    
        microsoft_earnings["Estimated EPS"][i]=microsoft_earnings["Estimated EPS"][i].replace("$","") 
        microsoft_earnings["Actual EPS"][i]=microsoft_earnings["Actual EPS"][i].replace("$","") 

In [158]:
for i in range(len(twitter_earnings["Estimated EPS"])):
    
        twitter_earnings["Estimated EPS"][i]=twitter_earnings["Estimated EPS"][i].replace("$","") 
        twitter_earnings["Actual EPS"][i]=twitter_earnings["Actual EPS"][i].replace("$","") 

In [167]:
earnings_list=[apple_earnings,microsoft_earnings,twitter_earnings]
for i in earnings_list:
    cambiar_tipo_datos(i)

## Tablas price

Primero quedemos poner todos los valores como float a excepción de la fecha

In [8]:
prices_list=[apple_price,twitter_price,microsoft_price]

In [9]:
for i in prices_list:
    
    cambiar_tipo_datos2(i)

Ahora voy a añadir una columna que se llame price change que calcule como ha cambiado el precio respecto al precio de cierre del día anterior

In [10]:
col_price_change(prices_list)

Vamos a redondear los valores a dos puntos decimales

In [11]:
for j in range(len(prices_list)):
    
    columnas=prices_list[j].columns

    for i in columnas:
        
        if i!="Date":
        
            prices_list[j][i]=prices_list[j][i].round(decimals=2)  

## Tablas events

In [188]:
events_twitter=pd.DataFrame(twitter_price, columns = ['Date', 'Price_change'])
events_twitter=events_twitter[events_twitter["Price_change"].abs()>15]
events_twitter.reset_index(drop=True,inplace=True)
events_twitter['news_link']="unknown"

In [171]:
events_apple=pd.DataFrame(apple_price, columns = ['Date', 'Price_change'])
events_apple=events_apple[events_apple["Price_change"].abs()>7]
events_apple.reset_index(drop=True,inplace=True)
events_apple['news_link']="unknown"

In [172]:
events_microsoft=pd.DataFrame(microsoft_price, columns = ['Date', 'Price_change'])
events_microsoft=events_microsoft[events_microsoft["Price_change"].abs()>7]
events_microsoft.reset_index(drop=True,inplace=True)
events_microsoft['news_link']="unknown"

Añadimos el link para una acción concreta en una fecha dada usando la función get_news_link:

In [174]:
for i in range(len(events_microsoft)):
    
    events_microsoft['news_link'][i]=get_news_link("MSFT",events_microsoft['Date'][i])
    

In [190]:
for i in range(len(events_twitter)):
    
    events_twitter['news_link'][i]=get_news_link("TWTR",events_twitter['Date'][i])

In [177]:
for i in range(len(events_apple)):
    
    events_apple['news_link'][i]=get_news_link("AAPL",events_apple['Date'][i])

Hacemos que los links sean clickables

In [93]:
#events_microsoft=events_microsoft.style.format({'news_link': make_clickable})

In [106]:
#events_twitter=events_twitter.style.format({'news_link': make_clickable})

In [114]:
#events_apple=events_apple.style.format({'news_link': make_clickable})

Exportamos los csv:

In [169]:
apple_price.to_csv("../data/new_data/apple_price.csv",index=False)
twitter_price.to_csv("../data/new_data/twitter_price.csv",index=False)
microsoft_price.to_csv("../data/new_data/microsoft_price.csv",index=False)
apple_earnings.to_csv("../data/new_data/apple_earnings.csv",index=False)
twitter_earnings.to_csv("../data/new_data/twitter_earnings.csv",index=False)
microsoft_earnings.to_csv("../data/new_data/microsoft_earnings.csv",index=False)

In [193]:
events_apple.to_csv("../data/new_data/events_apple.csv",index=False)
events_twitter.to_csv("../data/new_data/events_twitter.csv",index=False)
events_microsoft.to_csv("../data/new_data/events_microsoft.csv",index=False)

In [ ]:
#pip install openpyxl

In [ ]:
#conda install -c anaconda openpyxl

lst_df=Parallel(n_jobs=8, verbose=True)(delayed(extraer_links)(url) for url in equipos_stats_urls[:2])
        

def extraer links(data):
    lista=[]
    events=pd.DataFrame(data, columns = ['Date', 'Price_change'])
    events=events[events["Price_change"].abs()>7]
    events.reset_index(drop=True,inplace=True)
    events['news_link']="unknown"

    for i in range(len(events)):
    
        events['news_link'][i]=get_news_link("MSFT",events['Date'][i])
    
    